In [1]:
import os
import requests
import json
from pprint import pprint
from api_key import api_key
import pandas as pd

In [2]:
from xmljson import parker, Parker
from xml.etree.ElementTree import fromstring
from json import dumps

from xmljson import BadgerFish as bf


# dumps(parker.data(fromstring('<x><a>1</a><b>2</b></x>')))
# '{"a": 1, "b": 2}'
# dumps(parker.data(fromstring('<x><a>1</a><b>2</b></x>'), preserve_root=True))
# '{"x": {"a": 1, "b": 2}}'

In [3]:
# File to Load 
exchanges_df = pd.read_csv("Resources/Exchanges.csv")
exchanges_df.head()


,Unnamed: 0,Id,Name,Url,time,volume,exchange,date
0,0,2431,Bitstamp,https://www.bitstamp.net/,1580083200,1.334957e+08,Bitstamp,2020-01-27
1,1,2436,OKCoin,https://www.okcoin.com,1580083200,7.908577e+06,OKCoin,2020-01-27
2,2,2438,Poloniex,https://poloniex.com,1580083200,8.486261e+07,Poloniex,2020-01-27
3,3,2439,Kraken,https://r.kraken.com/z37qM,1580083200,1.556806e+08,Kraken,2020-01-27
4,4,2440,Bitfinex,https://bitfinex.com/?refcode=-Dkl7lkv,1580083200,1.384511e+08,Bitfinex,2020-01-27


In [4]:
#exchanges_df['Url']

In [101]:
exchanges_new = exchanges_df.drop(index=40)
#exchanges_new

In [102]:
# Save config information
url = "https://awis.api.alexa.com/api?"

# Documentation: https://awis.alexa.com/developer-guide
# See api_doc.txt for a summary

#Actions = [UrlInfo, TrafficHistory, CategoryBrowse, CategoryListings, SitesLinkingIn]

path_url = exchanges_new['Url']


In [103]:
headers = {"Accept":"application/json", "x-api-key":api_key }

In [104]:
# Build query URL for Action = UrlInfo, ResponseGroup = TrafficData

parser = bf(dict_type=dict)
response_traffic = []

for path in path_url:
        query_url = url + f'Action=urlInfo&ResponseGroup=TrafficData&Url='+path
        response_json = requests.get(query_url, headers=headers)
        response = parser.data(fromstring(response_json.content))
        response_traffic.append(response)


In [75]:
#Rank
response_traffic[32]['Awis']['Results']['Result']['Alexa']['TrafficData']['Rank']['$']

50819

In [76]:
#Page views per million
response_traffic[32]['Awis']['Results']['Result']['Alexa']['TrafficData']['UsageStatistics']['UsageStatistic'][0]['PageViews']['PerMillion']['Value']['$']

1.4

In [77]:
#Page views per user
response_traffic[32]['Awis']['Results']['Result']['Alexa']['TrafficData']['UsageStatistics']['UsageStatistic'][0]['PageViews']['PerUser']['Value']['$']

4.3

In [105]:
rank_list = []
page_views_permillion_list = []
page_views_peruser_list = []
successes = []

# iterate over the response list, each corresponding to one exchange

for x in range(len(path_url)):
    try:
        rank_list.append(
                response_traffic[x]['Awis']['Results']['Result']['Alexa']['TrafficData']['Rank']['$'])
        page_views_permillion_list.append(
                response_traffic[x]['Awis']['Results']['Result']['Alexa']['TrafficData']['UsageStatistics']['UsageStatistic'][0]['PageViews']['PerMillion']['Value']['$'])
        page_views_peruser_list.append(
                response_traffic[x]['Awis']['Results']['Result']['Alexa']['TrafficData']['UsageStatistics']['UsageStatistic'][0]['PageViews']['PerUser']['Value']['$'])
        successes.append(x)
    except KeyError:
        continue
        
        
print(len(rank_list))
print(len(page_views_permillion_list))
print(len(page_views_peruser_list))

43
43
43


In [106]:
len(path_url)

43

In [116]:
traffic_data_dict = {'url': path_url,
               'rank': rank_list,
               'page_views_per_million': page_views_permillion_list,
                'page_views_per_user': page_views_peruser_list
             }

traffic_data_df = pd.DataFrame(traffic_data_dict)
traffic_data_df

,url,rank,page_views_per_million,page_views_per_user
0,https://www.bitstamp.net/,17541,3.22,3.10
1,https://www.okcoin.com,174009,0.27,3.10
2,https://poloniex.com,6865,9.92,4.18
3,https://r.kraken.com/z37qM,7226,10.20,4.60
4,https://bitfinex.com/?refcode=-Dkl7lkv,7994,6.92,3.10
5,https://www.coinbase.com/join/59dba386c59cdf00...,2034,33.80,4.85
6,https://localbitcoins.com/?ch=irs1,5278,24.80,10.10
7,https://www.itbit.com,378252,0.09,2.40
8,https://hitbtc.com/?ref_id=5a155b131ea65,14757,4.23,3.60
9,https://www.lakebtc.com/?ref=glqg26,402635,0.10,3.20


In [108]:
traffic_data_df.to_csv('Resources/traffic_data.csv')

In [123]:
# Build query URL for Action = UrlInfo, ResponseGroup = RankByCountry

path=path_url

parser = bf(dict_type=dict)
response_rankcountry = []


for path in path_url:
        query_url = url + f'Action=urlInfo&ResponseGroup=RankByCountry&Url='+path
        response_json = requests.get(query_url, headers=headers)
        response = parser.data(fromstring(response_json.content))
        response_rankcountry.append(response)


In [143]:
#response_rankcountry[0]

In [142]:
#Country
response_rankcountry[0]['Awis']['Results']['Result']['Alexa']['TrafficData']['RankByCountry']['Country'][0]['@Code']

'VN'

In [144]:
#Visitors by country [0] (% from total)
response_rankcountry[0]['Awis']['Results']['Result']['Alexa']['TrafficData']['RankByCountry']['Country'][0]['Contribution']['Users']['$']

'0.7%'

In [164]:
list_of_dfs = []

for url_index in range(len(path_url)):
    url_list = []
    countries_list = []
    visitors_list = []


    for x in range(0,196):
        try:
            result_1 = response_rankcountry[url_index]['Awis']['Results']['Result']['Alexa']['TrafficData']['RankByCountry']['Country'][x]['@Code']
            result_2 = response_rankcountry[url_index]['Awis']['Results']['Result']['Alexa']['TrafficData']['RankByCountry']['Country'][x]['Contribution']['Users']['$']
        except (IndexError, KeyError):
            continue
            
        url_list.append(path_url.to_list()[url_index])
        countries_list.append(result_1)
        visitors_list.append(result_2)
    
    
    new_df = pd.DataFrame(zip(url_list, countries_list, visitors_list), columns=['url', 'countries', 'visitors'])
    list_of_dfs.append(new_df)



In [169]:
list_of_dfs[3]

,url,countries,visitors
0,https://r.kraken.com/z37qM,IN,2.0%
1,https://r.kraken.com/z37qM,BG,2.5%
2,https://r.kraken.com/z37qM,TR,1.1%
3,https://r.kraken.com/z37qM,IL,0.8%
4,https://r.kraken.com/z37qM,US,20.7%
5,https://r.kraken.com/z37qM,HK,0.8%
6,https://r.kraken.com/z37qM,IE,4.0%
7,https://r.kraken.com/z37qM,IR,0.5%
8,https://r.kraken.com/z37qM,BE,5.0%
9,https://r.kraken.com/z37qM,CH,4.3%


In [170]:
for st in range(0,43):
    filename = "Resources/visitor_per_country"+str(st)+".csv"
    list_of_dfs[st].to_csv(filename, index=False, header=True)